In [1]:
from meb import utils, datasets, core, models

from tqdm import tqdm
from functools import partial

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
import pytorchvideo
import timm
from timm.scheduler.cosine_lr import CosineLRScheduler
import pytorchvideo.transforms
import torch.nn.functional as F



pd.set_option("display.max_columns", 50)
%load_ext autoreload
%autoreload 2

# 1

In [13]:
c = datasets.CrossDataset(resize=64, ignore_validation=True, optical_flow=True)
df = c.data_frame
data = c.data

In [14]:
c = datasets.Samm(resize=64, ignore_validation=True, optical_flow=True)
dfs = c.data_frame
datas = c.data

In [15]:
# Add missing AU columns to SAMM that are missing compared to all data
dfs_aus = pd.DataFrame(dfs.loc[:, "AU1":].to_dict(), columns=df.loc[:, "AU1":].columns).fillna(0)
dfs_dropped = dfs.drop(dfs.loc[0, "AU1":].index.tolist(), axis=1)
dfs = pd.concat([dfs_dropped, dfs_aus], axis=1)

In [16]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

action_units = dfs.loc[:, "AU1":].columns
df_dataset = dfs
model = GradientBoostingClassifier(random_state=0)

aus = np.array(df_dataset.loc[:, action_units])
emotions = df_dataset["Objective Classes"].astype("str")
model.fit(aus, emotions)
predicted_emotions = model.predict(aus)
f1 = f1_score(emotions, predicted_emotions, average="macro")
print(f1 * 100)

99.17754854925882


In [17]:
df.insert(7, "objective_class", model.predict(np.array(df.loc[:, "AU1":])))

#### MEGC2018 validation

In [18]:
idx = df["objective_class"].isin(["6", "7"])
df = df[~idx].reset_index(drop=True)
data = data[~idx]

In [19]:
df["subject"] = df["subject"].astype(str) + df["dataset"]

In [20]:
from meb.core.train_eval import InputData
from typing import List

class MEGC2018Validator(core.Validator):
    def __init__(self, config: core.Config, verbose: bool = True):
        super().__init__(config, split_column="subject")
        self.verbose = verbose
        self.disable_tqdm = False

    def validate(
        self, df: pd.DataFrame, input_data: InputData, seed_n: int = 1
    ) -> List[torch.tensor]:
        utils.set_random_seeds(seed_n)
        subject_names = df.loc[df["dataset"].isin(["samm", "casme2"]), "subject"].unique()
        le = LabelEncoder()
        labels = le.fit_transform(df["objective_class"])
        outputs_list = []
        for subject_name in subject_names:
            train_metrics, test_metrics, outputs_test = self.validate_split(
                df, input_data, labels, subject_name
            )
            outputs_list.append(outputs_test)
            if self.verbose:
                self.printer.print_train_test_evaluation(
                    train_metrics, test_metrics, subject_name, outputs_test.shape[0]
                )

        # Calculate total f1-scores
        predictions = torch.cat(outputs_list)
        f1_total = self.evaluation_fn(labels[df["dataset"].isin(["samm", "casme2"])], predictions)
        if self.verbose:
            print("Total f1: {}".format(f1_total))
        return outputs_list

In [21]:
class Config(core.Config):
    action_units = None
    epochs = 200
    batch_size = 256
    evaluation_fn = utils.MultiClassF1Score
    criterion = nn.CrossEntropyLoss
    device = torch.device("cuda:1")
    optimizer = partial(optim.Adam, lr=1e-4, weight_decay=1e-3)
    model = partial(models.SSSNet, num_classes=5)

In [31]:
out = MEGC2018Validator(Config).validate(df, data)

100%|█████████████████████████████████████████| 200/200 [01:51<00:00,  1.79it/s]


Subject: 01casme2, n=8 | train_mean: 1.0 | test_mean: 0.6111


100%|█████████████████████████████████████████| 200/200 [01:50<00:00,  1.82it/s]


Subject: 02casme2, n=7 | train_mean: 1.0 | test_mean: 0.3333


100%|█████████████████████████████████████████| 200/200 [01:51<00:00,  1.80it/s]


...

Subject: 035samm, n=3 | train_mean: 1.0 | test_mean: 1.0


100%|█████████████████████████████████████████| 200/200 [01:51<00:00,  1.79it/s]


Subject: 036samm, n=1 | train_mean: 1.0 | test_mean: 0.0


100%|█████████████████████████████████████████| 200/200 [01:51<00:00,  1.79it/s]


Subject: 037samm, n=1 | train_mean: 1.0 | test_mean: 0.0
Total f1: [0.6934895631306097]


In [19]:
idx = df["dataset"].isin(["samm", "casme2"])
out = MEGC2018Validator(Config).validate(df[idx].reset_index(), data[idx])

100%|█████████████████████████████████████████| 200/200 [01:03<00:00,  3.17it/s]


Subject: 01casme2, n=8 | train_mean: 1.0 | test_mean: 1.0


100%|█████████████████████████████████████████| 200/200 [01:03<00:00,  3.14it/s]


Subject: 02casme2, n=7 | train_mean: 1.0 | test_mean: 0.3667


100%|█████████████████████████████████████████| 200/200 [01:03<00:00,  3.13it/s]


...

Subject: 035samm, n=3 | train_mean: 1.0 | test_mean: 1.0


100%|█████████████████████████████████████████| 200/200 [01:05<00:00,  3.06it/s]


Subject: 036samm, n=1 | train_mean: 1.0 | test_mean: 0.0


100%|█████████████████████████████████████████| 200/200 [01:05<00:00,  3.07it/s]


Subject: 037samm, n=1 | train_mean: 1.0 | test_mean: 0.0
Total f1: [0.6983133765880812]


In [21]:
class Config(core.Config):
    action_units = None
    epochs = 200
    batch_size = 32
    evaluation_fn = utils.MultiClassF1Score
    criterion = nn.CrossEntropyLoss
    device = torch.device("cuda:0")
    optimizer = partial(optim.Adam, lr=1e-4, weight_decay=1e-3)
    model = partial(timm.models.resnet18, num_classes=5, pretrained=True)

In [22]:
out = MEGC2018Validator(Config).validate(df, data)

100%|█████████████████████████████████████████| 200/200 [06:11<00:00,  1.86s/it]


Subject: 01casme2, n=8 | train_mean: 1.0 | test_mean: 0.6111


100%|█████████████████████████████████████████| 200/200 [06:09<00:00,  1.85s/it]


Subject: 02casme2, n=7 | train_mean: 1.0 | test_mean: 0.3333


100%|█████████████████████████████████████████| 200/200 [06:11<00:00,  1.86s/it]


...

Subject: 035samm, n=3 | train_mean: 1.0 | test_mean: 1.0


100%|█████████████████████████████████████████| 200/200 [06:06<00:00,  1.83s/it]


Subject: 036samm, n=1 | train_mean: 1.0 | test_mean: 0.0


100%|█████████████████████████████████████████| 200/200 [06:09<00:00,  1.85s/it]


Subject: 037samm, n=1 | train_mean: 1.0 | test_mean: 0.0
Total f1: [0.6319072169848342]


In [23]:
idx = df["dataset"].isin(["samm", "casme2"])
out = MEGC2018Validator(Config).validate(df[idx].reset_index(), data[idx])

100%|█████████████████████████████████████████| 200/200 [01:56<00:00,  1.72it/s]


Subject: 01casme2, n=8 | train_mean: 1.0 | test_mean: 0.6111


100%|█████████████████████████████████████████| 200/200 [01:56<00:00,  1.71it/s]


Subject: 02casme2, n=7 | train_mean: 1.0 | test_mean: 0.3667


100%|█████████████████████████████████████████| 200/200 [01:56<00:00,  1.72it/s]


...

Subject: 035samm, n=3 | train_mean: 1.0 | test_mean: 1.0


100%|█████████████████████████████████████████| 200/200 [02:09<00:00,  1.55it/s]


Subject: 036samm, n=1 | train_mean: 1.0 | test_mean: 0.0


100%|█████████████████████████████████████████| 200/200 [02:08<00:00,  1.56it/s]


Subject: 037samm, n=1 | train_mean: 1.0 | test_mean: 1.0
Total f1: [0.49944624713528907]


In [24]:
class Config(core.Config):
    action_units = None
    epochs = 200
    batch_size = 32
    evaluation_fn = utils.MultiClassF1Score
    criterion = nn.CrossEntropyLoss
    device = torch.device("cuda:0")
    optimizer = partial(optim.Adam, lr=1e-4, weight_decay=1e-3)
    model = partial(timm.models.resnet34, num_classes=5, pretrained=True)

In [25]:
out = MEGC2018Validator(Config).validate(df, data)

100%|█████████████████████████████████████████| 200/200 [10:42<00:00,  3.21s/it]


Subject: 01casme2, n=8 | train_mean: 1.0 | test_mean: 0.6111


100%|█████████████████████████████████████████| 200/200 [10:53<00:00,  3.27s/it]


Subject: 02casme2, n=7 | train_mean: 1.0 | test_mean: 0.5167


100%|█████████████████████████████████████████| 200/200 [10:51<00:00,  3.26s/it]


...

Subject: 035samm, n=3 | train_mean: 0.9251 | test_mean: 1.0


100%|█████████████████████████████████████████| 200/200 [12:02<00:00,  3.61s/it]


Subject: 036samm, n=1 | train_mean: 1.0 | test_mean: 1.0


100%|█████████████████████████████████████████| 200/200 [12:01<00:00,  3.61s/it]


Subject: 037samm, n=1 | train_mean: 1.0 | test_mean: 0.0
Total f1: [0.6818972235754047]


In [26]:
idx = df["dataset"].isin(["samm", "casme2"])
out = MEGC2018Validator(Config).validate(df[idx].reset_index(), data[idx])

100%|█████████████████████████████████████████| 200/200 [03:07<00:00,  1.07it/s]


Subject: 01casme2, n=8 | train_mean: 1.0 | test_mean: 1.0


100%|█████████████████████████████████████████| 200/200 [03:07<00:00,  1.07it/s]


Subject: 02casme2, n=7 | train_mean: 1.0 | test_mean: 0.5595


100%|█████████████████████████████████████████| 200/200 [03:06<00:00,  1.07it/s]


...

Subject: 035samm, n=3 | train_mean: 0.9961 | test_mean: 1.0


100%|█████████████████████████████████████████| 200/200 [03:22<00:00,  1.01s/it]


Subject: 036samm, n=1 | train_mean: 1.0 | test_mean: 0.0


100%|█████████████████████████████████████████| 200/200 [03:23<00:00,  1.02s/it]


Subject: 037samm, n=1 | train_mean: 1.0 | test_mean: 0.0
Total f1: [0.5763878968128576]


In [27]:
class Config(core.Config):
    action_units = None
    epochs = 200
    batch_size = 32
    evaluation_fn = utils.MultiClassF1Score
    criterion = nn.CrossEntropyLoss
    device = torch.device("cuda:0")
    optimizer = partial(optim.Adam, lr=1e-4, weight_decay=1e-3)
    model = partial(timm.models.resnet50, num_classes=5, pretrained=True)

In [28]:
out = MEGC2018Validator(Config).validate(df, data)

100%|█████████████████████████████████████████| 200/200 [16:12<00:00,  4.86s/it]


Subject: 01casme2, n=8 | train_mean: 1.0 | test_mean: 0.6111


100%|█████████████████████████████████████████| 200/200 [16:06<00:00,  4.83s/it]


Subject: 02casme2, n=7 | train_mean: 0.9988 | test_mean: 0.5167


100%|█████████████████████████████████████████| 200/200 [16:09<00:00,  4.85s/it]


...

Subject: 035samm, n=3 | train_mean: 0.9956 | test_mean: 1.0


100%|█████████████████████████████████████████| 200/200 [16:01<00:00,  4.81s/it]


Subject: 036samm, n=1 | train_mean: 1.0 | test_mean: 0.0


100%|█████████████████████████████████████████| 200/200 [15:48<00:00,  4.74s/it]


Subject: 037samm, n=1 | train_mean: 0.9935 | test_mean: 0.0
Total f1: [0.6466664611855174]


In [29]:
idx = df["dataset"].isin(["samm", "casme2"])
out = MEGC2018Validator(Config).validate(df[idx].reset_index(), data[idx])

100%|█████████████████████████████████████████| 200/200 [03:52<00:00,  1.16s/it]


Subject: 01casme2, n=8 | train_mean: 1.0 | test_mean: 0.6111


100%|█████████████████████████████████████████| 200/200 [03:50<00:00,  1.15s/it]


Subject: 02casme2, n=7 | train_mean: 1.0 | test_mean: 0.4333


100%|█████████████████████████████████████████| 200/200 [03:51<00:00,  1.16s/it]


...

Subject: 035samm, n=3 | train_mean: 1.0 | test_mean: 1.0


100%|█████████████████████████████████████████| 200/200 [04:13<00:00,  1.27s/it]


Subject: 036samm, n=1 | train_mean: 1.0 | test_mean: 1.0


100%|█████████████████████████████████████████| 200/200 [04:13<00:00,  1.27s/it]


Subject: 037samm, n=1 | train_mean: 1.0 | test_mean: 1.0
Total f1: [0.4348693101840503]


## R(2+1)D

In [23]:
c = datasets.CrossDataset(resize=112, ignore_validation=True, preload=True, color=True)
df = c.data_frame
data = c.data

Loading data: 100%|███████████████████████████| 860/860 [02:42<00:00,  5.29it/s]


In [24]:
c = datasets.Samm(resize=112, ignore_validation=True, preload=True, color=True)
dfs = c.data_frame
datas = c.data

Loading data: 100%|███████████████████████████| 159/159 [01:22<00:00,  1.94it/s]


In [25]:
# Add missing AU columns to SAMM that are missing compared to all data
dfs_aus = pd.DataFrame(dfs.loc[:, "AU1":].to_dict(), columns=df.loc[:, "AU1":].columns).fillna(0)
dfs_dropped = dfs.drop(dfs.loc[0, "AU1":].index.tolist(), axis=1)
dfs = pd.concat([dfs_dropped, dfs_aus], axis=1)

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

action_units = dfs.loc[:, "AU1":].columns
df_dataset = dfs
model = GradientBoostingClassifier(random_state=0)

aus = np.array(df_dataset.loc[:, action_units])
emotions = df_dataset["Objective Classes"].astype("str")
model.fit(aus, emotions)
predicted_emotions = model.predict(aus)
f1 = f1_score(emotions, predicted_emotions, average="macro")
print(f1 * 100)

99.17754854925882


In [27]:
df.insert(7, "objective_class", model.predict(np.array(df.loc[:, "AU1":])))

In [28]:
from torchvision import transforms
import torch.nn.functional as F
#interpolate samples with less than 8 frames
n_frames = 8
for i, video in enumerate(data):
    if video.shape[0] < n_frames:
        new_shape = (n_frames,) + video.shape[1:-1]
        video = torch.tensor(video).permute(3, 0, 1, 2).unsqueeze(0).float()
        new_video = F.interpolate(video, size=new_shape, mode="trilinear")
        data[i] = new_video.squeeze(0).permute(1, 2, 3, 0).byte().numpy()

In [29]:
#idx = df["dataset"].isin(["samm", "casme2"]) & (df["objective_class"].isin(["6", "7"]))
idx = df["objective_class"].isin(["6", "7"])
df = df[~idx].reset_index(drop=True)
data = data[~idx]

In [30]:
df["subject"] = df["subject"].astype(str) + df["dataset"]

In [31]:
# Create a function that returns the model as it needs to be modified
def r2plus1d(num_classes: int):
    model = torchvision.models.video.r2plus1d_18(weights=torchvision.models.video.R2Plus1D_18_Weights.DEFAULT)
    model.fc = nn.Linear(in_features=512, out_features=num_classes)
    return model

In [32]:
class Config(core.Config):
    action_units = None
    epochs = 100
    batch_size = 32
    evaluation_fn = utils.MultiClassF1Score
    criterion = nn.CrossEntropyLoss
    device = torch.device("cuda:1")
    optimizer = partial(optim.Adam, lr=1e-4, weight_decay=1e-3)
    model = partial(r2plus1d, num_classes=5)
    loss_scaler = torch.cuda.amp.GradScaler
    channels_last = torch.channels_last_3d
    train_transform = {
        "spatial": None,
        "temporal": datasets.UniformTemporalSubsample(8)
    }
    test_transform = {
        "spatial": None,
        "temporal": datasets.UniformTemporalSubsample(8)
    }

In [33]:
out = MEGC2018Validator(Config).validate(df, data)

100%|███████████████████████████████████████| 100/100 [1:19:15<00:00, 47.56s/it]


Subject: 01casme2, n=8 | train_mean: 0.9611 | test_mean: 0.4646


100%|███████████████████████████████████████| 100/100 [1:19:25<00:00, 47.65s/it]


Subject: 02casme2, n=7 | train_mean: 0.9981 | test_mean: 0.4167


100%|███████████████████████████████████████| 100/100 [1:19:23<00:00, 47.64s/it]


...

Subject: 035samm, n=3 | train_mean: 0.9985 | test_mean: 1.0


100%|███████████████████████████████████████| 100/100 [1:19:41<00:00, 47.81s/it]


Subject: 036samm, n=1 | train_mean: 1.0 | test_mean: 0.0


100%|███████████████████████████████████████| 100/100 [1:19:39<00:00, 47.80s/it]


Subject: 037samm, n=1 | train_mean: 0.945 | test_mean: 0.0
Total f1: [0.4822672328704776]


In [34]:
idx = df["dataset"].isin(["samm", "casme2"])
out = MEGC2018Validator(Config).validate(df[idx].reset_index(), data[idx])

100%|█████████████████████████████████████████| 100/100 [16:03<00:00,  9.63s/it]


Subject: 01casme2, n=8 | train_mean: 1.0 | test_mean: 0.2564


100%|█████████████████████████████████████████| 100/100 [16:06<00:00,  9.67s/it]


Subject: 02casme2, n=7 | train_mean: 1.0 | test_mean: 0.07143


100%|█████████████████████████████████████████| 100/100 [16:06<00:00,  9.66s/it]


...

Subject: 035samm, n=3 | train_mean: 1.0 | test_mean: 0.0


100%|█████████████████████████████████████████| 100/100 [16:34<00:00,  9.95s/it]


Subject: 036samm, n=1 | train_mean: 0.9848 | test_mean: 0.0


100%|█████████████████████████████████████████| 100/100 [16:35<00:00,  9.95s/it]


Subject: 037samm, n=1 | train_mean: 0.9923 | test_mean: 1.0
Total f1: [0.24785964912280703]
